In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
data_dir = 'drive/MyDrive/kaggle_mnist/digit-recognizer/'

for dirname, _, filenames in os.walk('drive/MyDrive/kaggle_mnist/digit-recognizer'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

drive/MyDrive/kaggle_mnist/digit-recognizer/test.csv
drive/MyDrive/kaggle_mnist/digit-recognizer/sample_submission.csv
drive/MyDrive/kaggle_mnist/digit-recognizer/train.csv


In [ ]:
train_data = pd.read_csv(data_dir + 'train.csv')
test_data = pd.read_csv(data_dir + 'test.csv')

In [ ]:
train_images = train_data.drop('label', axis=1).values.reshape(-1, 1, 28, 28)
test_images = test_data.values.reshape(-1, 1, 28, 28)
train_labels = train_data['label'].values

# nomalization
train_images = (train_images / 255.0).astype(np.float32)
test_images = (test_images / 255.0).astype(np.float32)

print(train_images.shape)
print(test_images.shape)

(42000, 1, 28, 28)
(28000, 1, 28, 28)


In [ ]:
train_images, valid_images, train_labels, valid_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42, stratify=train_labels)
print(type(train_images[0][0][0][0]))
print(type(valid_images[0][0][0][0]))

train_tensor = torch.utils.data.TensorDataset(torch.tensor(train_images), torch.tensor(train_labels))
valid_tensor = torch.utils.data.TensorDataset(torch.tensor(valid_images), torch.tensor(valid_labels))
test_tensor = torch.utils.data.TensorDataset(torch.tensor(test_images))

train_loader = DataLoader(train_tensor, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_tensor, batch_size=64, shuffle=False)
test_loader = DataLoader(test_tensor, batch_size=100, shuffle=False)

<class 'numpy.float32'>
<class 'numpy.float32'>


In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),  # (batch, 1, 28, 28)  ->  (batch, 32, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                                          # (batch, 32, 28, 28)  ->  (batch, 32, 14, 14)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1), # (batch, 32, 14, 14)  ->  (batch, 64, 14, 14)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                                          # (batch, 64, 14, 14)  ->  (batch, 64, 7, 7)
            nn.Flatten()
        )
        self.layer3 = nn.Sequential(
            nn.Linear(in_features=7*7*64, out_features=128),
            nn.Linear(in_features=128, out_features=10),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [ ]:
model = Network()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

Network(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Flatten(start_dim=1, end_dim=-1)
  )
  (layer3): Sequential(
    (0): Linear(in_features=3136, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=10, bias=True)
    (2): Softmax(dim=None)
  )
)

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
num_epochs = 100

for epoch in range(1, num_epochs + 1):

    train_loss = 0
    valid_loss = 0

    # train
    model.train()
    for step in range(len(train_loader)):
        images, labels = next(iter(train_loader))

        predictions = model(images)

        optimizer.zero_grad()
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss

    train_loss = train_loss / (step - 1)
    print("epoch: {}  train_loss: {:.5f}".format(epoch, train_loss))

    # validation
    model.eval()
    with torch.no_grad():
        for step in range(len(valid_loader)):
            images, labels = next(iter(valid_loader))
            predictions = model(images)
            loss = criterion(predictions, labels)

            valid_loss += loss

    valid_loss = valid_loss / (step - 1)
    print("epoch: {}  valid_loss: {:.5f}".format(epoch, valid_loss))
    print()


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch: 1  train_loss: 1.50767
epoch: 1  valid_loss: 1.50416

epoch: 2  train_loss: 1.50343
epoch: 2  valid_loss: 1.50869

epoch: 3  train_loss: 1.49930
epoch: 3  valid_loss: 1.49922

epoch: 4  train_loss: 1.49552
epoch: 4  valid_loss: 1.49649

epoch: 5  train_loss: 1.49222
epoch: 5  valid_loss: 1.50732

epoch: 6  train_loss: 1.48956
epoch: 6  valid_loss: 1.49762

epoch: 7  train_loss: 1.48838
epoch: 7  valid_loss: 1.50742

epoch: 8  train_loss: 1.48542
epoch: 8  valid_loss: 1.51085

epoch: 9  train_loss: 1.48616
epoch: 9  valid_loss: 1.51260

epoch: 10  train_loss: 1.48416
epoch: 10  valid_loss: 1.50298

epoch: 11  train_loss: 1.48393
epoch: 11  valid_loss: 1.50311

epoch: 12  train_loss: 1.48267
epoch: 12  valid_loss: 1.50864

epoch: 13  train_loss: 1.48097
epoch: 13  valid_loss: 1.49432

epoch: 14  train_loss: 1.47960
epoch: 14  valid_loss: 1.49350

epoch: 15  train_loss: 1.47886
epoch: 15  valid_loss: 1.49742

epoch: 16  train_loss: 1.47969
epoch: 16  valid_loss: 1.49308

epoch: 17 

In [ ]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x), axis=1)[:, None]

In [ ]:
test_pred = []
# softmax = nn.Softmax(dim=1)
with torch.no_grad():
    for step in range(len(test_loader)):
        images = next(iter(test_loader))

        predictions = model(images[0])

        test_pred.append(softmax(predictions.detach().cpu().numpy()))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [ ]:
submission = pd.read_csv(data_dir + 'sample_submission.csv')
print(np.concatenate(test_pred, axis=0).shape)
submission["Label"] = np.argmax(np.concatenate(test_pred, axis=0), axis=1)

(28000, 10)


In [ ]:
submission.head(10)

,ImageId,Label
0,1,1
1,2,5
2,3,1
3,4,1
4,5,1
5,6,1
6,7,1
7,8,1
8,9,1
9,10,2


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_1, *['ImageId'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_2, *['Label'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_3, *['index'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(_df_6, *[[['index', 'ImageId'], ['ImageId', 'Label']]], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_7, *['index', 'Label', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_8, *['index', 'count()', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_9, *['ImageId', 'Label', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_10, *['ImageId', 'count()', None], **{})
chart